# Clear then setup database

In [1]:
from dotenv.main import load_dotenv
from pymongo import MongoClient
import os

load_dotenv('./.env.local')

MongoClient(os.environ.get('MONGO_URI')).drop_database(
    os.environ.get('DB_NAME'))


In [2]:
from typing import OrderedDict
from pymongo import ASCENDING, MongoClient
from pymongo.errors import CollectionInvalid

load_dotenv('./.env.local')
client = MongoClient(os.environ.get('MONGO_URI'))
db = client['buffalographics']

collections = ['clients', 'projects', 'files']
schemas = {
    'clients':
    OrderedDict({
        'collMod': 'clients',
        'validator': {
            '$jsonSchema': {
                'bsonType': 'object',
                'properties': {
                    'name': {
                        'bsonType': 'string',
                        'description': 'Company Name'
                    },
                    'directory': {
                        'bsonType': 'string',
                        'description': 'Path to client folder'
                    }
                },
                'required': ['name', 'directory']
            }
        }
    }),
    'projects':
    OrderedDict({
        'collMod': 'projects',
        'validator': {
            '$jsonSchema': {
                'bsonType': 'object',
                'properties': {
                    'name': {
                        'bsonType': 'string',
                        'description': 'Project Name'
                    },
                    'client': {
                        'bsonType': 'string',
                        'description': 'Company Name'
                    },
                    'directory': {
                        'bsonType': 'string',
                        'description': 'Path to project folder'
                    }
                },
                'required': ['name', 'client', 'directory']
            }
        }
    })
}

for col in collections:

    try:
        db.create_collection(col)
        if schemas.get(col) is not None:
            db.command(schemas[col])

    except CollectionInvalid as err:
        # print(err)
        pass

db.clients.create_index([
    ('name', ASCENDING),
], unique=True)

db.projects.create_index([('name', ASCENDING), ('client', ASCENDING)],
                         unique=True)

db.files.create_index([('path', ASCENDING)], unique=True)

db.list_collection_names()


['clients', 'projects', 'files']

In [3]:
p = 'Bell Concrete/mixer/PRINT/bell_cc_drum_graphic_qty_2.pdf'

s = p.replace(' ','_').replace('/','-').replace('.pdf',
                                                '.jpg').lower()
print(s)

bell_concrete-mixer-print-bell_cc_drum_graphic_qty_2.jpg
